In [1]:
import numpy as np
import pandas as pd
import re
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential   # sequential model intializes the neural network
from keras.layers import Dense   # dense model builds the layers of our ANN 
from keras.layers import Dropout # to prevent overfitting
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
length = len(data['text'])
print(length)

7613


In [4]:
duplicates = data.duplicated().sum()
duplicates

0

In [5]:
# Since they are all NaN I will remove them.
data.drop(['location'], axis = 1, inplace = True)
data.drop(['keyword'], axis = 1, inplace = True)

In [6]:
data.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# preprocessing the text
def cleaner(old_text):
    cleaned_text = [] 
    for sentences in old_text :
        hold = re.sub('[^a-zA-Z]',' ', sentences)
        hold = hold.lower()
        cleaned_text.append(hold)
    return cleaned_text

In [8]:
data['cleaned_text'] = cleaner(data.text)
data.head()

,id,text,target,cleaned_text
0,1,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake m...
1,4,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are ...
3,6,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation or...
4,7,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as ...


In [9]:
x = data['cleaned_text']
y = data['target']

In [10]:
# using Tfidf Vectorizer to  vectorize the text
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(stop_words='english', binary = True)
X = tf.fit_transform(x)

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [12]:
def The_classifier():
    
    classifier = Sequential()

    classifier.add(Dense(output_dim = 100,init = 'uniform', activation = 'relu', input_dim = 21826))
    classifier.add(Dropout(p = 0.3))

    classifier.add(Dense(output_dim = 50,init = 'uniform', activation = 'relu'))
    classifier.add(Dropout(p = 0.3))

    classifier.add(Dense(output_dim = 25,init = 'uniform', activation = 'relu'))
    classifier.add(Dropout(p = 0.3))
    
    classifier.add(Dense(output_dim = 1,init = 'uniform', activation = 'sigmoid'))

    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']  )
    
    return classifier


In [13]:
classifier = KerasClassifier(build_fn = The_classifier)

In [14]:
classifier.fit(x_train,y_train, batch_size = 10, nb_epoch = 10)

W1229 01:31:52.637252  8208 deprecation_wrapper.py:119] From C:\Users\Kenechi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

C:\Users\Kenechi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=21826, units=100, kernel_initializer="uniform")`
  """
W1229 01:31:52.685288  8208 deprecation_wrapper.py:119] From C:\Users\Kenechi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1229 01:31:52.690290  8208 deprecation_wrapper.py:119] From C:\Users\Kenechi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

C:\Users\Kenechi\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWar

Epoch 1/10
6090/6090 [==============================] - 76s 12ms/step - loss: 0.5701 - acc: 0.6928
Epoch 2/10
6090/6090 [==============================] - 72s 12ms/step - loss: 0.2816 - acc: 0.8920
Epoch 3/10
6090/6090 [==============================] - 63s 10ms/step - loss: 0.1191 - acc: 0.9614 2s - loss: 0.1195 - acc: 0. - ETA: 2s - loss
Epoch 4/10
6090/6090 [==============================] - 52s 9ms/step - loss: 0.0536 - acc: 0.9839: 7s - loss: 0.0509 - acc:
Epoch 5/10
6090/6090 [==============================] - 55s 9ms/step - loss: 0.0301 - acc: 0.9913
Epoch 6/10
6090/6090 [==============================] - 46s 8ms/step - loss: 0.0287 - acc: 0.9913
Epoch 7/10
6090/6090 [==============================] - 46s 8ms/step - loss: 0.0205 - acc: 0.9931
Epoch 8/10
6090/6090 [==============================] - 45s 7ms/step - loss: 0.0182 - acc: 0.9939
Epoch 9/10
6090/6090 [==============================] - 45s 7ms/step - loss: 0.0169 - acc: 0.9941
Epoch 10/10
6090/6090 [=====================

In [15]:
y_pred = classifier.predict(x_test)

In [16]:
from sklearn.metrics import confusion_matrix, classification_report
check = confusion_matrix(y_test, y_pred)
print(check)

[[740 134]
 [230 419]]


In [17]:
check_1 = classification_report(y_test, y_pred)
print(check_1)

             precision    recall  f1-score   support

          0       0.76      0.85      0.80       874
          1       0.76      0.65      0.70       649

avg / total       0.76      0.76      0.76      1523



In [18]:
test = pd.read_csv('test.csv')
test['cleaned_text'] = cleaner(test.text)
X_t = tf.transform(test.cleaned_text)

In [19]:
test.head()

,id,keyword,location,text,cleaned_text
0,0,NaN,NaN,Just happened a terrible car crash,just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities s...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills in china and taiwan


In [20]:
#submit = pd.read_csv('sample_submission.csv')
#submit.head()

In [21]:
#submit['target'] = classifier.predict(X_t)
#submit.head(30)

In [22]:
#submit.to_csv("submission.csv", index=False)

In [ ]:
# Using Naive Bayes Classifier...................

In [23]:
from sklearn.naive_bayes import GaussianNB
classifier_1= GaussianNB()
classifier_1.fit(x_train.toarray(), y_train)

GaussianNB(priors=None)

In [24]:
y_pred = classifier_1.predict(x_test.toarray())

In [25]:
check_2 = classification_report(y_test, y_pred)
print(check_2)

             precision    recall  f1-score   support

          0       0.76      0.50      0.60       874
          1       0.54      0.78      0.64       649

avg / total       0.66      0.62      0.62      1523



In [ ]:
# Using Random Forest Classifier..............

In [26]:
from sklearn.ensemble import RandomForestClassifier
classifier_2 = RandomForestClassifier(n_estimators = 10, criterion= 'entropy' , random_state =0)
classifier_2.fit(x_train.toarray(), y_train)


C:\Users\Kenechi\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [27]:
y_pred = classifier_2.predict(x_test.toarray())

In [28]:
check_3 = classification_report(y_test, y_pred)
print(check_3)

             precision    recall  f1-score   support

          0       0.73      0.91      0.81       874
          1       0.82      0.55      0.66       649

avg / total       0.77      0.76      0.75      1523



In [ ]:
# For the competition I used the neutral network.